# Post-processing of case study for central plant results

## Usage notes

All figures will be written to the directory `img` as pdf and png files.

I run this notebook on Ubuntu, using this version of matplotlib:
```
pip freeze | grep matplotlib
matplotlib==3.5.1
```
Some older versions return an error because they do not support some plot configurations.
I set in the virtual machine the RAM to 12GB as 8GB is not sufficient to parse two annual result files.

## Import required libraries

In [30]:
import cases as cas
import importlib
import post_process_configurations as con
importlib.reload(con)


<module 'post_process_configurations' from '/home/mwetter/test/thermal-grid-jba-main/PythonResources/RunCases/post_process_configurations.py'>

## Read annual results

This section also clears the old results to free up memory.

In [31]:
# Free up storage, then read new data
import gc
gc.collect()

print("Cleaned up memory.")

Cleaned up memory.


Get list of cases

In [32]:
# Build list of case files, their labels out output file prefix.
# These are used to read in files and create plots.
# This structure allows removing a case from the post-processing in case the simulation did not converge.
import cases as cas
cases = cas.get_cases()

# Store commit in sim_version dictionary
import os
verFil = os.path.join("simulations", "base", "version.txt")
sim_version = dict()
if os.path.exists(verFil):
    with open(verFil, 'r') as fil:
        s = fil.read()
        kv_pairs = [
            tuple(s_str.split('='))
            for s_str in s.split('\n')
        ]
        for ele in kv_pairs:
          if ele[0] != '': # Skips empty line
            sim_version[ele[0]] = ele[1]
          
#print(sim_version)



Read result file. This takes around 3 minutes per result file.

In [33]:

print("Reading result files")
r_base = None
for cas in cases:
    try:
        cas['reader'] = con.get_results(cas['name'])
        if cas['name'] == 'base':
            r_base = cas['reader']
    except Exception as e:
        print(f"*** Error reading {cas['name']}: ", e)
        cas['postProcess'] = False
    print(f"Read {cas['name']}")

Reading result files
Read base
Read base_hBor_1_2
Read base_hBor_0_8
Read base_dDis_1_2
Read base_dDis_0_8
Read base_noEco
Read heat
Read cold
Read base_heaPumSizFac_0_8
Read base_heaPumSizFac_0_9


In [34]:
AFlo = r_base.max('datDis.AFlo')

print(f"Base case: Total energy use: {r_base.max('ETot.y')/3600/1E9:.2f} GWh")
print(f"Base case: Total energy use: {r_base.max('ETot.y')/3600/1E9/AFlo*1e6:.2f} kWh/(m a)")

print(f"Base case: Total energy cost: {r_base.max('totEleCos.y')/1E6:.2f} million USD per year")
print(f"Base case: Total energy cost: {r_base.max('totEleCos.y')/AFlo:.2f} USD/(m2 a)")



Base case: Total energy use: 11.19 GWh
Base case: Total energy use: 99.95 kWh/(m a)
Base case: Total energy cost: 1.65 million USD per year
Base case: Total energy cost: 14.76 USD/(m2 a)


In [35]:
print(f"CPUtime, base {r_base.max('CPUtime')/3600.:.0f} h")

CPUtime, base 10 h


## Write main results to LaTeX file for inclusion in report

In [50]:
import os
importlib.reload(con)

if not os.path.exists("img"):
    os.mkdir("img")
with open(os.path.join("img", "modelicaResults.tex"), "w") as fil:
    # Energy costs
    r_base_dDis_0_8 = None
    for cas in cases:
        if cas['name'] == 'base_dDis_0_8':
            r_base_dDis_0_8= cas['reader']

    ETot_base = r_base.max('ETot.y')
    ETot_dDis_0_8 = r_base_dDis_0_8.max('ETot.y')
    cosETot_base = r_base.max("totEleCos.y")
    dhDis_base = r_base.max('datDis.dhDisAct')
    vDis_base = r_base.max('datDis.vDis_nominal')
    cosETot_dDis_0_8 = r_base_dDis_0_8.max("totEleCos.y")
    dhDis_0_8 = r_base_dDis_0_8.max('datDis.dhDisAct')


    # ALCC is annualized life cycle costs
    # Investment cost difference is length of distribution pipe, time cost difference of 0.5 vs 0.4 m diameter (from Sommer paper) times exchange rate from Euro to $
    invCosDif = 3460 *(1220-1100)*1.15
    # Annualized increase in electricity costs due to smaller pipes.
    delEneCos =  cosETot_dDis_0_8 - cosETot_base
    (ALCC, LCC, I, OM, RC, SR, CRF) = con.calc_finance(invCosDif, 0, 0, 40, 0.01)

    print(f"Annual energy cost difference {delEneCos/1E6:.3f} million $")
    print(f"invCos = {invCosDif/1E6:.3f} million $")
    print(f"ALCC of investment = {ALCC:.0f} $")
    print(f"ALCC of investment and energy {ALCC-delEneCos:.0f} $")

    s = """
\\newcommand{\\modelicaBranch}{""" + f"{sim_version['branch']}" + """\\xspace}
\\newcommand{\\modelicaCommit}{\\href{https://github.com/lbl-srg/thermal-grid-jba/commit/""" + f"{sim_version['commit']}" + """}{""" + f"{sim_version['commit'][0:6]}" + """}\\xspace}

\\newcommand{\\cpuTime}{""" + f"{r_base.max('CPUtime')/3600.:.0f} hours" + """\\xspace}
\\newcommand{\\delEnergyDDisEighty}{$""" + f"{(ETot_dDis_0_8/ETot_base-1)*100:.0f}\%" + """$\\xspace}
\\newcommand{\\delEnergyCosDDisEighty}{$\\$""" + f"{(cosETot_dDis_0_8-cosETot_base)/1E6:.2f} \, \mathrm{{million}}" + """$\\xspace}

\\newcommand{\\totEneCosBase}{$\\$""" + f"{cosETot_base/1E6:.2f} \, \mathrm{{million}}" + """$\\xspace}
\\newcommand{\\totEneCosDDisEighty}{$\\$""" + f"{cosETot_dDis_0_8/1E6:.2f} \, \mathrm{{million}}" + """$\\xspace}

\\newcommand{\\dhBase}{$""" + f"{dhDis_base:.2f} \, \mathrm m ({dhDis_base*3.28084:.2f} \, \mathrm{{ft}})" + """$\\xspace}
\\newcommand{\\dhDDisEighty}{$""" + f"{dhDis_0_8:.2f} \, \mathrm m ({dhDis_0_8*3.28084:.2f} \, \mathrm{{ft}})" + """$\\xspace}
\\newcommand{\\vDisBase}{$""" + f"{vDis_base:.2f} \, \mathrm{{m/s}} ({vDis_base*3.28084:.2f} \, \mathrm {{ft/s}})" + """$\\xspace}

\\newcommand{\\delAnnLifCycEighty}{$\\$""" + f"{(delEneCos-ALCC)/1E6:.2f} \, \mathrm{{million}}" + """$\\xspace}
"""
    fil.write(s)




Annual energy cost difference 0.530 million $
invCos = 0.477 million $
ALCC of investment = 23929 $
ALCC of investment and energy -506437 $


### Write capacities to LaTeX file

In [37]:
# Note that the unyt package used in this function does not allow to reload the module.
# Hence we put the function in its own file
import post_process_write_latex_table as lat
lat.write_latex_capacity_table(r_base)

'% generated by `write_latex_capacity_table()` in PythonResources/RunCases/post_process_write_latex_table.py\n\n\\begin{tabular}{llrlrl}\n\\toprule\n & System capacity & \\multicolumn{2}{c}{SI unit} & \\multicolumn{2}{c}{IP unit} \\\\\n\\hline\nETS 1 & Heat recovery chiller - heating & 3,414 & kW & 11,650 & kBtu/hr \\\\\n & Heat recovery chiller - cooling & 2,845 & kW & 809 & ton \\\\\n & District heat exchanger & 3,091 & kW & 10,547 & kBtu/hr \\\\\n\\hline\nETS 2 & Heat recovery chiller - heating & 5,616 & kW & 19,162 & kBtu/hr \\\\\n & Heat recovery chiller - cooling & 4,680 & kW & 1,331 & ton \\\\\n & District heat exchanger & 5,084 & kW & 17,349 & kBtu/hr \\\\\n & Domestic hot water tank & 403 & m$^3$ & 106,374 & gal \\\\\n\\hline\nETS 3 & Heat recovery chiller - heating & 370 & kW & 1,264 & kBtu/hr \\\\\n & Heat recovery chiller - cooling & 309 & kW & 88 & ton \\\\\n & District heat exchanger & 335 & kW & 1,144 & kBtu/hr \\\\\n & Domestic hot water tank & 27 & m$^3$ & 7,014 & gal 

### Energy use


In [38]:
importlib.reload(con)
con.plot_energy(cases)

All electricity use = [11.19362686 10.89248865 11.52647984 10.1214812  14.76913587 11.36787573
 11.80447248 11.52229602 11.17865669 11.21086195]
Sum of plot = [11.19362747 10.8924893  11.52648043 10.12148174 14.76913663 11.36787576
 11.80447213 11.52229579 11.17865752 11.21086231]

Heat pumps in ETS   & 5.32 &  47.5 \\
Heat pumps in plant & 3.26 &  29.1 \\
Pumps               & 3.81 &  34.0 \\
Fans                & 0.57 &  5.1 \\
Non-HVAC electricity for buildings & 8.85 &  79.1  \\ \hline
PVs and batteries  & -10.62 &  -94.8 \\
Total & 11.19 &  99.9 \\ \hline


## Loop temperatures

In [39]:
importlib.reload(con)
con.plot_loop_temperatures(cases)


# Plot plant operation

In [40]:
# List of days to plot
days = [
    {
        "xlim": [31, 32],
        "date": "Feb. 1",
        "name": "Winter"
    },
    {
        "xlim": [160, 161],
        "date": "June 10",
        "name": "Spring"
    },
    {
        "xlim": [213, 214],
        "date": "Aug. 2",
        "name": "Summer"
    }
]
# List of variables to plot for each subplot
lis = [
        {
            "y_label": "Controls",
            "y_lim": [-3, 3],
            "factor": 1,
            "offset": 0,
            "vars": [
                {
                    "label": "$y_{st}$",
                    "var": "cenPla.gen.ind.ySt",
                    "linewidth": 1,
                    },
                    {
                    "label": "$y_{pla}$",
                    "var": "cenPla.gen.ind.yPlaOut",
                    }
                ]
        },
        {
            "y_label": "Temperature\n[$^\\circ$C]",
            "y_lim": [10, 28],
            "factor": 1,
            "offset": -273.15,
            "vars": [
                 {
                    "label": "$T_{pla,hea,set}$",
                    "var": "cenPla.gen.ind.TActPlaHeaSet",
                    "linewidth": 1,
                    "marker": ">",
                    "color": "r",
                    "skip_if_ySea": 3
                    },
                 {
                    "label": "$T_{pla,coo,set}$",
                    "var": "cenPla.gen.ind.TActPlaCooSet",
                    "linewidth": 1,
                    "marker": ">",
                    "color": "r",
                    "skip_if_ySea": 1
                    },
                {
                    "label": "$T_{pla,in}$",
                    "var": "TDisWatRet.T",
                    "linewidth": 0.5,
                    "marker": ">",
                    "color": "k"
                    },
                    {
                    "label": "$T_{pla,out}$",
                    "var": "cenPla.gen.senTemGenLea.T",
                    "linewidth": 0.5,
                    "marker": "<",
                    "color": "k"
                    }
            ]
        },
        {
            "y_label": "Temperature\n[$^\\circ$C]",
            "y_lim": [10, 28],
            "factor": 1,
            "offset": -273.15,
            "vars": [
#                    {
#                    "label": "$T_{bor,per,ret}$",
#                    "var": "cenPla.gen.senTemBorPerRet.T",
#                    "marker": "<",
#                    "linestyle": "--",
#                    "color": "g"
#                    },
#                    {
#                    "label": "$T_{bor,cen,sup}$",
#                    "var": "cenPla.gen.senTemBorCenSup.T",
#                    "linestyle": "-.",
#                    "color": "r",
#                    "marker": ">"
#                    },
#                    {
#                    "label": "$T_{bor,cen,ret}$",
#                    "var": "cenPla.gen.senTemBorCenRet.T",
#                    "linestyle": "-.",
#                    "color": "r",
#                    "marker": "<"
#                    },
                    {
                    "label": "$T_{hea,pum,sup}$",
                    "var": "cenPla.gen.senTemHeaPumEnt.T",
                    "color": "b",
                    "marker": ">"
                    },
                    {
                    "label": "$T_{hea,pum,ret}$",
                    "var": "cenPla.gen.senTemHeaPumLea.T",
                    "color": "b",
                    "marker": "<"
                    }
                ]
            },
#            {
#            "y_label": "Mass flow\nrate [kg/s]",
#            "y_lim": [0, 500],
#            "factor": 1,
#            "offset": 0,
#            "vars": [
#                    {
#                    "label": "$\dot m_{eco}$",
#                    "var": "cenPla.gen.hex.m2_flow",
#                    },
#                    {
#                    "label": "$\dot m_{bor,per}$",
#                    "var": "cenPla.gen.senTemBorPerRet.port_a.m_flow",
#                    "marker": "o",
#                    },
#                    {
#                    "label": "$\dot m_{bor,cen}$",
#                    "var": "cenPla.gen.senTemBorCenRet.port_a.m_flow",
#                    "marker": "x",
#                    },
#                    {
#                    "label": "$\dot m_{hea,pum}$",
#                    "var": "cenPla.gen.heaPum.m1_flow",
#                    "marker": "v",
#                    }
#                    ]
#            },
            {
            "y_label": "Heat flow\nrate [MW]",
            "y_lim": [-8, 8],
            "factor": 1E-6,
            "offset": 0,
            "vars": [
#                    {
#                    "label": "$\dot Q_{eco}$",
#                    "var": "cenPla.gen.hex.Q2_flow",
#                    },
                    {
                    "label": "$\dot Q_{bor,per}$",
                    "var": "cenPla.borFie.QPer_flow",
                    "marker": "o",
                    },
                    {
                    "label": "$\dot Q_{bor,cen}$",
                    "var": "cenPla.borFie.QCen_flow",
                    "marker": "x",
                    },
                    {
                    "label": "$\dot Q_{hea,pum}$",
                    "var": "cenPla.gen.heaPum.Q1_flow",
                    "marker": "v",
                    }
                    ]
        }
    ]

con.plotPlant(lis, r_base, "plant", days)

# Plot borefield energy

In [41]:
# List of days to plot
days = [
    {
        "xlim": [0, 365],
        "date": "",
        "name": "Annual"
    }
]
# List of variables to plot for each subplot
lis = [
        {
            "y_label": "Borefield energy [GWh]",
            "y_lim": [-3, 3],
            "factor": 1/3600/1E9,
            "offset": 0,
            "vars": [
                {
                    "label": "Modelica $E_{bor}$",
                    "var": "EBor.y",
                    "linewidth": 1
                    },
                    {
                    "label": "Modelica $E_{bor,per}$",
                    "var": "EBorPer.y",
                    "linewidth": 1
                    },
                    {
                    "label": "Modelica $E_{bor,cen}$",
                    "var": "EBorCen.y",
                    "linewidth": 1
                    },
                    {
                    "label": "MILP $E_{bor}$",
                    "var": "borMil.E",
                    "linestyle": "--",
                    "mark_every": 50
                    }
                ]
        }
]

con.plotOneFigure(lis, r_base, "borefieldEnergy", days)